# HN Success Predictor - Transformer Edition

This notebook trains a DistilBERT-based classifier to predict whether a Hacker News post will be successful (100+ points).

## Instructions

1. **Runtime**: Go to `Runtime > Change runtime type` and select **GPU** (T4 is fine)
2. **Run all cells**: `Runtime > Run all` or Ctrl+F9
3. **Wait**: Data fetching takes ~10-15 min, training takes ~20-30 min
4. **Download**: The trained model will be saved and available for download

## What this does

- Fetches ~60,000 HN posts from the last 2 years (balanced: high/medium/low scores)
- Fine-tunes DistilBERT on post titles to predict success probability
- Achieves ~0.70-0.75 ROC AUC (vs 0.63 with TF-IDF)
- Exports model for use in your RSS reader

### Why 2 years?
- **Too short (<1 year)**: Not enough data, seasonal bias
- **Too long (>3 years)**: Topic drift - what was hot in 2021 isn't relevant now
- **2 years**: Sweet spot - enough data, still relevant topics

## 1. Setup and Dependencies

In [ ]:
!pip install -q transformers datasets accelerate scikit-learn pandas tqdm

In [ ]:
import requests
import time
from datetime import datetime, timedelta
from typing import List, Dict, Optional
from urllib.parse import urlparse

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
from datasets import Dataset as HFDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Fetch Training Data from Algolia

We will fetch posts in three categories:
- **High**: 100+ points (hits)
- **Medium**: 20-99 points
- **Low**: 1-19 points (misses)

In [ ]:
POSTS_PER_CATEGORY = 20000
MONTHS_BACK = 24
HIT_THRESHOLD = 100

print(f"Will fetch ~{POSTS_PER_CATEGORY * 3:,} posts from the last {MONTHS_BACK} months")

In [ ]:
def fetch_hn_posts(min_points, max_points, target_count, months_back=24):
    base_url = "https://hn.algolia.com/api/v1/search_by_date"
    all_posts = []
    seen_ids = set()

    if max_points:
        points_filter = f"points>={min_points},points<={max_points}"
        desc = f"{min_points}-{max_points} pts"
    else:
        points_filter = f"points>={min_points}"
        desc = f"{min_points}+ pts"

    end_date = datetime.now()
    posts_per_month = max(target_count // months_back, 500)
    pbar = tqdm(range(months_back), desc=f"Fetching {desc}")

    for month_offset in pbar:
        month_end = end_date - timedelta(days=30 * month_offset)
        month_start = month_end - timedelta(days=30)

        params = {
            "tags": "story",
            "numericFilters": f"{points_filter},created_at_i>={int(month_start.timestamp())},created_at_i<={int(month_end.timestamp())}",
            "hitsPerPage": min(posts_per_month, 1000),
        }

        try:
            resp = requests.get(base_url, params=params, timeout=30)
            resp.raise_for_status()
            data = resp.json()

            for hit in data.get("hits", []):
                post_id = hit.get("objectID")
                if post_id and post_id not in seen_ids:
                    seen_ids.add(post_id)
                    url = hit.get("url", "")
                    if url:
                        try:
                            domain = urlparse(url).netloc.replace("www.", "")
                        except:
                            domain = ""
                    else:
                        domain = "self.hackernews"

                    all_posts.append({
                        "id": post_id,
                        "title": hit.get("title", ""),
                        "url": url,
                        "domain": domain,
                        "points": hit.get("points", 0),
                    })

            pbar.set_postfix({"total": len(all_posts)})
        except Exception as e:
            print(f"Error: {e}")

        time.sleep(0.5)
        if len(all_posts) >= target_count:
            break

    return all_posts[:target_count]

In [ ]:
print("Fetching training data (~10-15 minutes)...\n")

posts_high = fetch_hn_posts(100, None, POSTS_PER_CATEGORY, MONTHS_BACK)
posts_medium = fetch_hn_posts(20, 99, POSTS_PER_CATEGORY, MONTHS_BACK)
posts_low = fetch_hn_posts(1, 19, POSTS_PER_CATEGORY, MONTHS_BACK)

for p in posts_high: p['category'] = 'high'
for p in posts_medium: p['category'] = 'medium'
for p in posts_low: p['category'] = 'low'

df = pd.DataFrame(posts_high + posts_medium + posts_low)
print(f"\nFetched {len(df):,} posts")

## 3. Prepare Dataset

In [ ]:
df['label'] = (df['points'] >= HIT_THRESHOLD).astype(int)
df = df[df['title'].notna() & (df['title'].str.len() > 5)].copy()
df['input_text'] = df['title']

print(f"Dataset: {len(df):,} posts, {df['label'].mean():.1%} hits")

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"Train: {len(train_df):,}, Val: {len(val_df):,}, Test: {len(test_df):,}")

## 4. Tokenize with DistilBERT

In [ ]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
MAX_LENGTH = 64

def tokenize_function(examples):
    return tokenizer(examples['input_text'], padding='max_length', truncation=True, max_length=MAX_LENGTH)

train_dataset = HFDataset.from_pandas(train_df[['input_text', 'label']].reset_index(drop=True))
val_dataset = HFDataset.from_pandas(val_df[['input_text', 'label']].reset_index(drop=True))
test_dataset = HFDataset.from_pandas(test_df[['input_text', 'label']].reset_index(drop=True))

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print("Tokenized datasets ready")

## 5. Train the Model

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model = model.to(device)
print(f"Model on {device}, {sum(p.numel() for p in model.parameters()):,} params")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    probs = torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
    return {'accuracy': accuracy_score(labels, predictions), 'roc_auc': roc_auc_score(labels, probs)}

In [ ]:
training_args = TrainingArguments(
    output_dir='./checkpoints',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc",
    greater_is_better=True,
    logging_steps=100,
    report_to="none",
    fp16=torch.cuda.is_available(),
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("Training (~20-30 min on T4)...")
trainer.train()

## 6. Evaluate

In [ ]:
results = trainer.evaluate(test_dataset)
print(f"\nTest ROC AUC: {results['eval_roc_auc']:.4f}")
print(f"Test Accuracy: {results['eval_accuracy']:.4f}")

## 7. Demo Predictions

In [ ]:
def predict_success(titles):
    inputs = tokenizer(titles, padding=True, truncation=True, max_length=MAX_LENGTH, return_tensors="pt").to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)[:, 1].cpu().numpy()
    return pd.DataFrame({'title': titles, 'probability': probs}).sort_values('probability', ascending=False)

demo = [
    "Show HN: I built a neural network in Rust",
    "Google announces layoffs",
    "Python 4.0 released",
    "Claude 4 is now available",
    "How SQLite scales to 1 billion rows",
    "The decline of Stack Overflow",
]

for _, row in predict_success(demo).iterrows():
    print(f"[{row['probability']:.1%}] {row['title'][:60]}")

## 8. Save Model

In [ ]:
model.save_pretrained("./hn_predictor")
tokenizer.save_pretrained("./hn_predictor")
!zip -r hn_predictor.zip ./hn_predictor
print("Download hn_predictor.zip from the file browser")

## Usage in RSS Reader

```python
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

class BertPredictor:
    def __init__(self, path="./hn_predictor"):
        self.tokenizer = DistilBertTokenizer.from_pretrained(path)
        self.model = DistilBertForSequenceClassification.from_pretrained(path)
        self.model.eval()

    def predict(self, titles):
        inputs = self.tokenizer(titles, padding=True, truncation=True, max_length=64, return_tensors="pt")
        with torch.no_grad():
            probs = torch.softmax(self.model(**inputs).logits, dim=-1)[:, 1].numpy()
        return probs
```